In [1]:
import pandas as pd
import numpy as np
import os

In [4]:
root_path = os.path.dirname(os.path.abspath(os.path.curdir)) # 1 up from /notebooks (curdir) to main project folder
data_folder_path = os.path.join(root_path, 'wildfire_prediction', 'data')
data_file_path = os.path.join(data_folder_path, 'FH_data_final.csv')

data = pd.read_csv(data_file_path)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40971 entries, 0 to 40970
Data columns (total 35 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   Year                                                      40971 non-null  float64
 1   Month                                                     40971 non-null  float64
 2   Day                                                       40971 non-null  float64
 3   count()[unit: km^2]                                       40971 non-null  float64
 4   mean() Temperature                                        40971 non-null  float64
 5   max() Temperature                                         40971 non-null  float64
 6   min() Temperature                                         40971 non-null  float64
 7   mean() WindSpeed                                          40971 non-null  float64
 8   mean() RelativeH

In [6]:
y = data.target
X = data.drop(columns = 'target')

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

In [8]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)

In [13]:
X_train.shape[1]

34

In [66]:
from tensorflow.keras import Sequential, layers
model = Sequential()
model.add(layers.Dense(200, activation='relu', input_dim=X_train.shape[1]))
model.add(layers.Dense(150, activation='relu'))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(75, activation='relu'))
model.add(layers.Dense(25, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [67]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_42 (Dense)             (None, 200)               7000      
_________________________________________________________________
dense_43 (Dense)             (None, 150)               30150     
_________________________________________________________________
dense_44 (Dense)             (None, 100)               15100     
_________________________________________________________________
dense_45 (Dense)             (None, 75)                7575      
_________________________________________________________________
dense_46 (Dense)             (None, 25)                1900      
_________________________________________________________________
dense_47 (Dense)             (None, 10)                260       
_________________________________________________________________
dense_48 (Dense)             (None, 1)               

In [68]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy', 
    metrics = 'accuracy')

In [69]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience = 50, restore_best_weights=True)

model.fit(X_train, y_train, 
          batch_size=32, 
          epochs=500, 
          validation_split=0.3,
          callbacks=[es])

Epoch 1/500
673/673 [==============================] - 2s 2ms/step - loss: 0.4109 - accuracy: 0.8072 - val_loss: 0.3550 - val_accuracy: 0.8280
Epoch 2/500
673/673 [==============================] - 1s 2ms/step - loss: 0.3576 - accuracy: 0.8281 - val_loss: 0.3679 - val_accuracy: 0.8300
Epoch 3/500
673/673 [==============================] - 1s 2ms/step - loss: 0.3475 - accuracy: 0.8312 - val_loss: 0.3359 - val_accuracy: 0.8414
Epoch 4/500
673/673 [==============================] - 1s 2ms/step - loss: 0.3380 - accuracy: 0.8401 - val_loss: 0.3532 - val_accuracy: 0.8338
Epoch 5/500
673/673 [==============================] - 1s 2ms/step - loss: 0.3383 - accuracy: 0.8380 - val_loss: 0.3452 - val_accuracy: 0.8320
Epoch 6/500
673/673 [==============================] - 1s 2ms/step - loss: 0.3335 - accuracy: 0.8413 - val_loss: 0.3313 - val_accuracy: 0.8446
Epoch 7/500
673/673 [==============================] - 1s 2ms/step - loss: 0.3318 - accuracy: 0.8431 - val_loss: 0.3489 - val_accuracy: 0.8390

In [70]:
model.evaluate(scaler.transform(X_test), y_test)

321/321 [==============================] - 0s 800us/step - loss: 0.3337 - accuracy: 0.8392


[0.33367636799812317, 0.8392072916030884]